In [1]:
import csv
import pandas as pd
import numpy as np

In [32]:
data = pd.read_csv('data.csv')

In [3]:
data.head()

,raw_row_number,date,time,location,lat,lng,beat,subject_age,subject_race,subject_sex,...,contraband_weapons,search_conducted,search_basis,reason_for_stop,use_of_force_description,raw_subject_sdrace,raw_subject_resultofencounter,raw_subject_searchconducted,raw_subject_typeofsearch,raw_subject_resultofsearch
0,17213,NaN,01:33:00,E. 28th St. & Park BLVD,37.803084,-122.237247,NaN,NaN,asian/pacific islander,female,...,NaN,False,NaN,Traffic Violation,NaN,A,"Citation,",No,NaN,NaN
1,17214,NaN,14:48:00,1759 SEMINARY AV,37.767568,-122.196820,NaN,NaN,black,male,...,False,True,other,Probation/Parole,NaN,B,"FI Report,",Yes,"Prob./Parole,",NaN
2,1,2013-04-01,00:00:00,31st St And Mlk Jr Way,37.820599,-122.270734,NaN,NaN,white,male,...,False,True,other,Traffic Violation,NaN,W,"Warning,",Yes,"Prob./Parole,",NaN
3,2,2013-04-01,00:01:00,3000 Block Of San Pablo Ave,37.821246,-122.276488,NaN,NaN,black,male,...,NaN,False,NaN,Traffic Violation,NaN,B,"FI Report,",No,NaN,NaN
4,19,2013-04-01,10:41:00,12th St/Broadway,37.802937,-122.271738,NaN,NaN,hispanic,female,...,NaN,False,NaN,Probable Cause,NaN,H,"Citation,",No,NaN,NaN


In [5]:
set(data['subject_race'])

{'asian/pacific islander', 'black', 'hispanic', 'other', 'white'}

In [14]:
data.columns

Index(['raw_row_number', 'date', 'time', 'location', 'lat', 'lng', 'beat',
       'subject_age', 'subject_race', 'subject_sex', 'officer_assignment',
       'type', 'arrest_made', 'citation_issued', 'warning_issued', 'outcome',
       'contraband_found', 'contraband_drugs', 'contraband_weapons',
       'search_conducted', 'search_basis', 'reason_for_stop',
       'use_of_force_description', 'raw_subject_sdrace',
       'raw_subject_resultofencounter', 'raw_subject_searchconducted',
       'raw_subject_typeofsearch', 'raw_subject_resultofsearch', 'action'],
      dtype='object')

In [33]:
data['action'] = data['citation_issued'] | data['arrest_made']

In [34]:
inputs = ['subject_race', 'subject_sex', 'lat', 'lng', 'subject_age', 'reason_for_stop']
label = ['action']

In [35]:
data['subject_age'].median()

29.0

In [36]:
data['subject_age'] = data['subject_age'].fillna(data['subject_age'].median())

In [230]:
set(data['reason_for_stop'])

{'Consensual Encounter',
 'Consensual Encounter|Consensual Encounter|Consensual Encounter|Traffic Violation',
 'Consensual Encounter|Consensual Encounter|Probable Cause',
 'Consensual Encounter|Consensual Encounter|Probable Cause|Reasonable Suspicion',
 'Consensual Encounter|Consensual Encounter|Reasonable Suspicion|Reasonable Suspicion',
 'Consensual Encounter|Consensual Encounter|Traffic Violation',
 'Consensual Encounter|Probable Cause',
 'Consensual Encounter|Probable Cause|Consensual Encounter',
 'Consensual Encounter|Probable Cause|Consensual Encounter|Probable Cause',
 'Consensual Encounter|Probable Cause|Probable Cause|Probable Cause|Probable Cause',
 'Consensual Encounter|Probable Cause|Probation/Parole',
 'Consensual Encounter|Probation/Parole',
 'Consensual Encounter|Reasonable Suspicion',
 'Consensual Encounter|Reasonable Suspicion|Reasonable Suspicion',
 'Consensual Encounter|Traffic Violation',
 'Consensual Encounter|Traffic Violation|Traffic Violation',
 'Other-Consensua

In [120]:
data[inputs + label].head()

,subject_race,subject_sex,lat,lng,subject_age,reason_for_stop,action
0,asian/pacific islander,female,37.803084,-122.237247,29.0,Traffic Violation,True
1,black,male,37.767568,-122.196820,29.0,Probation/Parole,False
2,white,male,37.820599,-122.270734,29.0,Traffic Violation,False
3,black,male,37.821246,-122.276488,29.0,Traffic Violation,False
4,hispanic,female,37.802937,-122.271738,29.0,Probable Cause,True


In [119]:
races = list(set(data['subject_race']))
sexes = list(set(data['subject_sex']))
reasons = list(set(data['reason_for_stop']))

In [216]:
data['clean_lng'] = data['lng'] - data['lng'].mean() 
data['clean_lat'] = data['lat'] - data['lat'].mean()
data['clean_age'] = data['subject_age'] - data['subject_age'].mean()

In [217]:
data['race'] = data.apply(lambda row: races.index(row['subject_race']), axis=1)
data['sex']= data.apply(lambda row: sexes.index(row['subject_sex']), axis=1)
data['reason']= data.apply(lambda row: reasons.index(row['reason_for_stop']), axis=1)

In [306]:
inputs = ['race', 'sex', 'clean_lat', 'clean_lng', 'clean_age']#, 'reason']
inputs = ['race']
label = 'action'

## Load Data

In [307]:
import torch

In [308]:
data[inputs] = data[inputs].dropna()

In [309]:
inputs_npy = data[inputs].to_numpy()
labels_npy = data[label].to_numpy().astype('int')
data_npy = np.array(list(zip(inputs_npy, labels_npy)))

In [329]:
data_npy[:, 0]

array([array([3.]), array([4.]), array([1.]), ..., array([4.]),
       array([0.]), array([4.])], dtype=object)

In [316]:
inds = np.arange(len(data))
np.random.shuffle(inds)
train_inds = inds[:int(len(data)*0.8)]
test_inds = inds[int(len(data)*0.8):]
train_data = [list(x) for x in data_npy[train_inds]]
test_data = [list(x) for x in data_npy[test_inds]]

In [317]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128)

In [318]:
for x, y in train_loader:
    print(x)
    break

tensor([[4.],
        [1.],
        [4.],
        [4.],
        [1.],
        [4.],
        [3.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [3.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [0.],
        [3.],
        [3.],
        [2.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [0.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [2.],
        [4.],
        [0.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [3.],
        [3.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [0.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
      

## Models

In [319]:
import torch
import torch.nn.functional as F

In [338]:
linear_model = torch.nn.Sequential(torch.nn.Linear(len(inputs), 128),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(128, 128),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(128, 1))
                                  #torch.nn.Sigmoid())
linear_optim = torch.optim.Adam(linear_model.parameters(), lr=1e-4)

## Training

In [344]:
def train_model(model, optim, train_loader, test_loader, activation='sigmoid', n_epochs=20):
    train_losses = []
    val_losses = []
    for epoch in range(n_epochs):
        epoch_losses = []
        for x, y in train_loader:
            print(x)
            y_prime = model(x.float())
            print(y_prime.max(), y_prime.min())
            #y_prime = F.sigmoid(y_prime)
            #print(y_prime.max(), y_prime.min())
            loss = F.mse_loss(y_prime, y.float())
            print("LOSS", loss)
            optim.zero_grad()
            loss.backward()
            optim.step()
            epoch_losses.append(loss.item())
        train_losses.append(np.mean(epoch_losses))
        val_losses.append(evaluate_model(model, test_loader))
        print("EPOCH {}: Train Loss: {}, Val Loss: {}".format(epoch, train_losses[-1], val_losses[-1]))    
    return train_losses, val_losses

In [345]:
def evaluate_model(model, test_loader):
    with torch.no_grad():
        losses = []
        for x, y in test_loader:
            y_prime = model(x)
            loss = F.binary_cross_entropy(y_prime, y)
            losses.append(loss.item())
    return np.mean(losses)
        

In [347]:
t_loss, v_loss = train_model(linear_model, linear_optim, train_loader, test_loader)

tensor([[4.],
        [1.],
        [4.],
        [4.],
        [1.],
        [4.],
        [3.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [3.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [0.],
        [3.],
        [3.],
        [2.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [0.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [2.],
        [4.],
        [0.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [3.],
        [3.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [0.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
      

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':


tensor([[4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [0.],
        [1.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [2.],
        [1.],
        [0.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [0.],
        [4.],
        [0.],
        [0.],
        [4.],
        [1.],
        [4.],
        [4.],
        [3.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
      

tensor([[4.],
        [2.],
        [1.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [0.],
        [4.],
        [0.],
        [4.],
        [3.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [1.],
        [1.],
        [4.],
        [0.],
        [1.],
        [4.],
        [0.],
        [4.],
        [1.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [0.],
        [4.],
        [4.],
        [3.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [2.],
        [4.],
        [4.],
        [1.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
      

tensor([[4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [1.],
        [0.],
        [3.],
        [4.],
        [4.],
        [0.],
        [0.],
        [4.],
        [1.],
        [1.],
        [2.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [1.],
        [3.],
        [4.],
        [3.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [0.],
      

...

tensor([[4.],
        [4.],
        [2.],
        [4.],
        [0.],
        [4.],
        [1.],
        [4.],
        [1.],
        [4.],
        [0.],
        [0.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [1.],
        [3.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [0.],
        [0.],
        [0.],
        [4.],
        [1.],
        [4.],
        [4.],
        [0.],
        [1.],
        [4.],
        [4.],
        [3.],
        [4.],
        [0.],
        [4.],
        [2.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
      

tensor([[4.],
        [4.],
        [4.],
        [4.],
        [3.],
        [1.],
        [1.],
        [4.],
        [0.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [1.],
        [3.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [3.],
        [4.],
        [4.],
        [2.],
        [1.],
        [4.],
        [4.],
        [0.],
        [0.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [1.],
        [0.],
        [0.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [3.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [1.],
        [4.],
        [0.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [1.],
        [2.],
        [4.],
        [4.],
        [3.],
      

tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
LOSS tensor(nan, grad_fn=<MseLossBackward>)
tensor([[4.],
        [1.],
        [4.],
        [4.],
        [0.],
        [4.],
        [nan],
        [2.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [1.],
        [2.],
        [0.],
        [4.],
        [2.],
        [0.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [0.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [3.],
        [4.],
        [4.],
        [4.],
        [0.],
        [0.],
        [4.],
        [1.],
        [1.],
        [4.],
        [4.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [4.],
        [4.],
        [1.],
        [4.],
        [3.],
        [3.],
        [0.],
        [4.],
        [3.],
        [4.],
        [1.],
        [4.],
        [4.],
 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([101])) that is different to the input size (torch.Size([101, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':


RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm

AttributeError: module 'torch' has no attribute 'inf'